In [1]:
import pandas as pd
from selenium import webdriver 
from bs4 import BeautifulSoup
import time
import re

In [2]:
from datetime import date, timedelta, datetime
import os.path

In [3]:
t0 =  date.today()

In [4]:
#news_url = 'https://www.google.com/search?hl=en&gl=us&tbm=nws&authuser=0&q=徽文化'
news_url="https://www.google.com/#q=%E5%BE%BD%E6%96%87%E5%8C%96&tbm=nws&num=100"
datetime_str = t0
fname1 = './data/news/' + 'google-huizhou-news' + '.csv'


In [5]:
def get_columns():
    return ('dt', 'source', 'domain', 'url', 'title', 'summary', 'image', 'brief', 'body', 'raw_page', 'from') 
def get_empty_columns():
    return ('', '', '', '', '', '', '', '', '', '', '') 
def get_empty_columns1():
    return ('', '', '', '', '', '', '', '', '', '', '','') 



In [6]:
browser = webdriver.Firefox() 
#browser = webdriver.Chrome('/Users/nhu2000/Downloads/Chromedriver')

In [7]:
if os.path.isfile(fname1): 
    df = pd.read_csv(fname1)
    idx = len(df.index)
else:
    df = pd.DataFrame(columns = get_columns())
    idx = 0

In [8]:
print df.count()

dt          1252
source      1754
domain         0
url         1754
title       1754
summary        0
image       1376
brief       1754
body           0
raw_page       0
from        1754
dtype: int64


In [9]:

browser.get(news_url)
time.sleep(5)


In [10]:
df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2017-01-01,人民网,NaN,http://culture.people.com.cn/n1/2017/0101/c101...,徽州古建筑保护遇窘境专家建言防“建设性破坏”,NaN,NaN,安徽省文史研究馆馆员方利山近日在谈到徽州古建筑保护时表示。 徽派建筑是中国传统建筑最重要的流...,NaN,NaN,google
1,2016-12-09,中国新闻网,NaN,http://www.chinanews.com/cul/2016/12-10/808973...,30余位中外书画家皖南体验“徽文化”,NaN,NaN,中新社黄山12月10日电(赵强)“我对皖南一直很向往，'徽文化'作为中华传统文化重要组成部分...,NaN,NaN,google
2,2016-12-31,搜狐,NaN,http://travel.sohu.com/20161231/n477476185.shtml,石川坞-千年古徽杭宗亲的联络线,NaN,https://encrypted-tbn1.gstatic.com/images?q=tb...,"梓桐镇是一座历史文化古镇,受到千年古徽州文化的影响,其文化底蕴颇为深厚。 因此来到这里旅行首...",NaN,NaN,google
3,2016-12-31,搜狐,NaN,http://sports.sohu.com/20161231/n477491596.shtml,叶长荫：竞与赛把人气聚学与鉴中促宣传,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",作为徽商故里、徽文化的发祥地黄山市不仅文化灿烂，而且生态优质，森林覆盖率高达82.9%，地表...,NaN,NaN,google
4,2017-01-01,中国网,NaN,http://travel.china.com.cn/txt/2017-01/01/cont...,名山名园携手美丽黄山时隔55年再入颐和园,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",2017年1月1日至2月12日（从元旦到元宵）， “梦幻黄山礼仪徽州”摄影作品 ... 象征...,NaN,NaN,google


In [11]:
#print browser.page_source

In [28]:
soup = BeautifulSoup(browser.page_source)
sub_texts = soup.findAll('div', {'class':'g'})

#print len(sub_texts)
for txt in sub_texts:

    #set title and url    
    link = txt.find('div', {'class': '_cnc'})
    
    print link.a.text
    print link.a['href']
    
    #check if it is already existed
    t0 = abs(sum(df['url'].str.find(str(link.a['href'])))) 
    if(t0 == len(df)):   
        print 'add -------'
        print link.a.text
        df.loc[idx] = get_empty_columns()

        #set title and url    
        df.loc[idx]['url'] = link.a['href']
        df.loc[idx]['title'] = link.a.text

        #set image source
        if txt.a.img:
            df.loc[idx]['image'] = txt.a.img['src'] 
        
        #split between source and time with '-'
        t1 = link.div.text
        t2 = t1.split('-')
        
        l0 = len(t2)
        
        #print t3.text
        ss = re.findall(ur'(\d+)年(\d+)月(\d+)日',t2[l0-1])
        if len(ss) == 0:
            #print t4[1].encode('utf-8')
            t30 = t2[l0-1]
            t40 = t30.split(' ')
            if 'ago' in t2[l0-1] or u'天之前' in t2[l0-1]:
                date_N_days_ago = datetime.now() - timedelta(days=int(t40[0]))
                t5 = date_N_days_ago.date()
                #t6 = t4[1]
            elif(len(t40) == 1):
                t5 = t0
                #t6 = t4[0]
            else:
                t5 = ''
                #t6 = t40[0]
            #print t5, t6
        else:
            ss = re.findall(ur'(\d+)年(\d+)月(\d+)日',t2[l0-1])
            t5 = str(ss[0][0]) + '-' + str(ss[0][1]) + '-' + str(ss[0][2])
            #t6 = t4[1]



        #if "ago" in t2[1]:
         #   df.loc[idx]['dt'] = t0
        #else:                    
          #  df.loc[idx]['dt'] = pd.to_datetime(t2[l0-1]).to_pydatetime().date()
    
        df.loc[idx]['source'] = t2[0]
    
        t3 = link.find('div', {'class': 'st'})
        df.loc[idx]['brief'] = t3.text

        #print txt, t2
        df.loc[idx]['from'] = 'google'
            
        #load_one_page(df, idx, link.a['href'])
        idx += 1 


徽州新桃花源记
http://js.ifeng.com/travel/lx/detail_2014_02/18/1863887_0.shtml
众泰T600众行中国问茶之道探寻歙县古镇
http://www.autohome.com.cn/info/201503/865864.html
老干妈不上市不融资专家:陶华碧有"土办法"
http://money.163.com/16/0906/07/C08TSDKJ002580S6.html
add -------
老干妈不上市不融资专家:陶华碧有"土办法"
姚振华背后的“潮汕商帮” ：11人坐拥6000亿
http://news.xinhuanet.com/fortune/2016-07/12/c_129135906.htm
add -------
姚振华背后的“潮汕商帮” ：11人坐拥6000亿
台船员遭海盗绑架五年获释人质画面曝光
http://www.ntdtv.com/xtr/gb/2016/10/25/a1293186.html
add -------
台船员遭海盗绑架五年获释人质画面曝光
“先锋书店”20年坚守与突围
http://society.people.com.cn/n1/2016/0812/c226879-28631374.html
add -------
“先锋书店”20年坚守与突围
黄金骑士队！赌城维加斯的NHL新球队有名字了
http://sports.qq.com/a/20161123/037641.htm
add -------
黄金骑士队！赌城维加斯的NHL新球队有名字了
笑蜀專欄：文化淪亡比國家淪亡更可怕——從徽州地名之爭說起
http://www.storm.mg/article/103733
元旦小长假部分北京市民选择离京避霾
http://www.chinanews.com/sh/2017/01-02/8111318.shtml
索马里海盗释放26人质含9名中国船员
http://www.ntdtv.com/xtr/gb/2016/10/24/a1293144.html
add -------
索马里海盗释放26人质含9名中国船员
清代官场文化与拜年习俗——王振忠说门簿
http://sh.xinhuanet.com/2017-01/28/c_136018031.htm
add 

In [29]:
print (len(df))

2049


In [30]:
df.to_csv(fname1, encoding='utf-8', index=False)

In [31]:
if(browser):
    browser.quit()

In [32]:
df.head(100)

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2017-01-01,人民网,NaN,http://culture.people.com.cn/n1/2017/0101/c101...,徽州古建筑保护遇窘境专家建言防“建设性破坏”,NaN,NaN,安徽省文史研究馆馆员方利山近日在谈到徽州古建筑保护时表示。 徽派建筑是中国传统建筑最重要的流...,NaN,NaN,google
1,2016-12-09,中国新闻网,NaN,http://www.chinanews.com/cul/2016/12-10/808973...,30余位中外书画家皖南体验“徽文化”,NaN,NaN,中新社黄山12月10日电(赵强)“我对皖南一直很向往，'徽文化'作为中华传统文化重要组成部分...,NaN,NaN,google
2,2016-12-31,搜狐,NaN,http://travel.sohu.com/20161231/n477476185.shtml,石川坞-千年古徽杭宗亲的联络线,NaN,https://encrypted-tbn1.gstatic.com/images?q=tb...,"梓桐镇是一座历史文化古镇,受到千年古徽州文化的影响,其文化底蕴颇为深厚。 因此来到这里旅行首...",NaN,NaN,google
3,2016-12-31,搜狐,NaN,http://sports.sohu.com/20161231/n477491596.shtml,叶长荫：竞与赛把人气聚学与鉴中促宣传,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",作为徽商故里、徽文化的发祥地黄山市不仅文化灿烂，而且生态优质，森林覆盖率高达82.9%，地表...,NaN,NaN,google
4,2017-01-01,中国网,NaN,http://travel.china.com.cn/txt/2017-01/01/cont...,名山名园携手美丽黄山时隔55年再入颐和园,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",2017年1月1日至2月12日（从元旦到元宵）， “梦幻黄山礼仪徽州”摄影作品 ... 象征...,NaN,NaN,google
5,2016-12-27,搜狐,NaN,http://auto.sohu.com/20161228/n477152703.shtml,驾新创酷游历徽州，找回年轻时的节奏感,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",上汽通用旗下分属于两个品牌的大型自驾活动很引人关注。一个是开先河的别克寰行中国，已历三年，开...,NaN,NaN,google
6,2016-12-08,东方财富网,NaN,"http://finance.eastmoney.com/news/1349,2016120...",黄山旅游：旅游与徽文化融合,NaN,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",【黄山旅游：旅游与徽文化融合】华安证券认为，黄山旅游成为黄山市乃至安徽省旅游资源的运作平台，...,NaN,NaN,google
7,2016-12-18,中国网,NaN,http://travel.china.com.cn/txt/2016-12/19/cont...,徽派建筑：中华大地上的一抹亮色(图),NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",古往今来，安徽文化自成一派，徽商、徽雕、徽剧、徽菜、徽派建筑、新安艺术，每一宗都源远流长。其...,NaN,NaN,google
8,2016-12-26,中国网,NaN,http://travel.china.com.cn/txt/2016-12/27/cont...,黄山市搭建古民居产权平台交易额超700万,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",黄山市是国家级徽州文化生态保护区的核心区，现有6000多幢徽派古民居遍布各地，是活态传承徽文...,NaN,NaN,google
9,2016-12-29,搜狐,NaN,http://sports.sohu.com/20161230/n477354009.shtml,一场厦马经济效益5.55亿2020年全国赛事800+场,NaN,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",今年4月在徽州举办的徽马规模只有1000人，但它产生的经济效应绝不是这个数字所能代表。“徽州...,NaN,NaN,google
